In [1]:
!pip install kiwipiepy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.2 MB/s eta 0:00:00
  Created wheel for kiwipiepy_model: filename=kiwipiepy_model-0.20.0-py3-none-any.whl size=34818026 sha256=218817eb3878924669acc91d7088ddd73cd1f6e9c938967564ba0239be81d0f0
  Stored in directory: /root/.cache/pip/wheels/ca/c8/52/3a539d6e9065b191fe1c215e0203dcc3e00601c0e3d3d39824
Successfully built kiwipiepy_model


In [2]:
import pandas as pd
from kiwipiepy import Kiwi
import kiwipiepy
import os

# 형태소 분석하기

In [ ]:
import pandas as pd
from kiwipiepy import Kiwi
import os

# 1. CSV 파일 경로 설정 (실제 파일 경로로 수정하세요)
merged_df_file = '/content/2025-01-21-12_final_lower.csv'          # 입력 CSV 파일 경로
user_dictionary_file = '/content/technicalTags (3).txt'   # 사용자 사전 파일 경로
output_file = '/content/[lower]analyzed_job_data_final.csv'            # 출력 CSV 파일 경로

# 2. CSV 파일 읽기
try:
    merged_df = pd.read_csv(merged_df_file)
    print("CSV 파일 읽기 완료.")
except FileNotFoundError:
    print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인하세요: {merged_df_file}")
    exit(1)

# 3. Kiwi 초기화 및 사용자 사전 적용
# kiwi = Kiwi(typos='basic_with_continual_and_lengthening')  # 기본 오타 정보, 연철, 장음화 함께 사용
kiwi = Kiwi(typos='basic_with_continual')

# 3.1. 사용자 사전 파일에서 단어 읽기
try:
    with open(user_dictionary_file, 'r', encoding='utf-8') as f:
        user_custom_words = [line.strip() for line in f if line.strip()]
    print(f"사용자 사전 단어 수: {len(user_custom_words)}개")
except FileNotFoundError:
    print(f"오류: 사용자 사전 파일을 찾을 수 없습니다. 경로를 확인하세요: {user_dictionary_file}")
    exit(1)

# 3.2. 사용자 사전 단어를 NNP 품사로 등록
for word in user_custom_words:
    kiwi.add_user_word(word, 'SL')  # IT 용어는 보통 고유 명사(NNP)로 분류

print("사용자 사전 단어 등록 완료.")

In [ ]:

# 5. 형태소 분석 함수 정의
allowed_pos_tags = ['SL']  # 외래어만 추출

def analyze_foreign_words(text):
    if not isinstance(text, str):
        return []
    # 형태소 분석 수행 (불용어 제거 및 IT 용어 고정)
    tokens = kiwi.tokenize(text, normalize_coda=True, split_complex=True)
    # 외래어(SL)만 추출
    foreign_words = [morph for morph, pos, _, _ in tokens if pos == 'SL']
    # 중복된 토큰 제거 (순서 유지)
    seen = set()
    unique_foreign_words = []
    for morph in foreign_words:
        if morph not in seen:
            seen.add(morph)
            unique_foreign_words.append(morph)
    return unique_foreign_words

def analyze_foreign_words_all(text):
    if not isinstance(text, str):
        return []
    # 형태소 분석 수행 (불용어 제거 및 IT 용어 고정)
    tokens = kiwi.tokenize(text, normalize_coda=True,  split_complex=True)
    combined_tokens = []
    morph_seen = set()
    i = 0
    while i < len(tokens):
        morph, pos, _, _ = tokens[i]
        # 현재 토큰이 SN이고 다음 토큰이 NNB인 경우 결합 (외래어 포함)
        if pos == 'SN' and (i + 1) < len(tokens):
            next_morph, next_pos, _, _ = tokens[i + 1]
            if next_pos == 'NNB':
                combined_morph = morph + next_morph
                combined_pos = 'SN+NNB'
                if pos == 'SL' and combined_morph not in morph_seen:
                    combined_tokens.append(f"{combined_morph}+{combined_pos}")
                    morph_seen.add(combined_morph)
                i += 2
                continue
        # 외래어(SL)만 처리
        if pos == 'SL' and morph not in morph_seen:
            combined_tokens.append(f"{morph}+{pos}")
            morph_seen.add(morph)
        i += 1
    return combined_tokens


In [ ]:
# 6. 필요한 열이 존재하는지 확인하고, 없으면 빈 문자열로 채움
required_columns = ['description', 'requirement', 'preferredExperience']
for col in required_columns:
    if col not in merged_df.columns:
        merged_df[col] = ''

# 7. 'description'과 'requirement'를 합쳐 새로운 열 생성
merged_df['description_requirement'] = (
    merged_df['description'].fillna('') + ' ' +
    merged_df['requirement'].fillna('')
)

# 8. 'preferredExperience'는 별도로 새로운 열 생성 (필요시 다른 전처리 추가 가능)
merged_df['preferredExperience_cleaned'] = merged_df['preferredExperience'].fillna('')

# 9. 형태소 분석 적용
print("형태소 분석 시작...")
merged_df['combined_analyzed'] = merged_df['description_requirement'].apply(analyze_text)
merged_df['combined_analyzed_all'] = merged_df['description_requirement'].apply(analyze_text_all)
merged_df['preferredExperience_analyzed'] = merged_df['preferredExperience_cleaned'].apply(analyze_text)
merged_df['preferredExperience_analyzed_all'] = merged_df['preferredExperience_cleaned'].apply(analyze_text_all)
print("형태소 분석 완료.")
print('-'*80)

In [ ]:
# 7.5. 결과를 새로운 CSV 파일에 저장
output_columns = [
    'id',
    'description',
    'requirement',
    'preferredExperience',
    'description_requirement',  # 'description+requirement' -> 'description_requirement'으로 변경
    'combined_analyzed',  # 'description_requirement_analyzed' -> 'combined_analyzed'으로 변경
    'combined_analyzed_all',  # 'description_requirement_analyzed_all' -> 'combined_analyzed_all'으로 변경
    'preferredExperience_analyzed',
    'preferredExperience_analyzed_all'
]
# 필요한 열만 선택하여 저장 (존재하지 않는 열은 제외)
existing_output_columns = [col for col in output_columns if col in merged_df.columns]

# 출력 디렉토리가 존재하지 않으면 생성
output_dir = os.path.dirname(output_file)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)

# CSV 파일로 저장
merged_df[existing_output_columns].to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"형태소 분석 결과가 '{output_file}' 파일에 저장되었습니다.")


In [ ]:

# 5. 청크 크기 설정
chunksize = 10000  # 필요에 따라 조정

# 6. 출력 파일 초기화 (헤더 포함)
if os.path.exists(output_file):
    os.remove(output_file)

# 7. CSV 파일 청크 단위로 읽기 및 처리
for chunk_number, chunk in enumerate(pd.read_csv(merged_df_file, chunksize=chunksize), start=1):
    print(f"청크 {chunk_number} 처리 시작...")

    # 7.1. 필요한 열이 존재하는지 확인하고, 없으면 빈 문자열로 채움
    required_columns = ['id', 'description', 'requirement', 'preferredExperience']
    for col in required_columns:
        if col not in chunk.columns:
            chunk[col] = ''

    # 7.2. 'description'과 'requirement'를 합쳐 새로운 열 생성
    chunk['description+requirement'] = (
        chunk['description'].fillna('') + ' ' +
        chunk['requirement'].fillna('')
    )

    # 7.3. 'preferredExperience'는 별도로 새로운 열 생성
    chunk['preferredExperience_cleaned'] = chunk['preferredExperience'].fillna('')

    # 7.4. 형태소 분석 적용
    chunk['description_requirement_analyzed'] = chunk['description+requirement'].apply(analyze_text_with_custom_words)
    chunk['description_requirement_analyzed_all'] = chunk['description+requirement'].apply(analyze_text_all_with_custom_words)
    chunk['preferredExperience_analyzed'] = chunk['preferredExperience_cleaned'].apply(analyze_text_with_custom_words)
    chunk['preferredExperience_analyzed_all'] = chunk['preferredExperience_cleaned'].apply(analyze_text_all_with_custom_words)

    print("형태소 분석 완료.")
    print('-'*80)
